In [ ]:
%load_ext autoreload

# 引入必要的库

In [ ]:
%autoreload
import sys,os
import numpy as np
import matplotlib.pyplot as plt
from tianmoucv.isp import fourdirection2xy,poisson_blend
from tianmoucv.data import TianmoucDataReader
import torch.nn.functional as F
import torch
import cv2

train='/data/tianmoucData/tianmoucReconDataset_merged/train/'
dirlist = os.listdir(train)
traindata = [train + e for e in dirlist]

val='/data/tianmoucData/tianmoucReconDataset_merged/test/'
vallist = os.listdir(val)
valdata = [val + e for e in vallist]
key_list = []

print('---------------------------------------------------')
for sampleset in traindata:
    print('---->',sampleset,'有：',len(os.listdir(sampleset)),'个样本')
    for e in os.listdir(sampleset):
        print(e)
        key_list.append(e)
print('---------------------------------------------------')
for sampleset in valdata:
    print('---->',sampleset,'有：',len(os.listdir(sampleset)),'个样本')
    for e in os.listdir(sampleset):
        print(e)
        key_list.append(e)
        
all_data = valdata + traindata

# 融合图像

In [ ]:
%autoreload
import torch.nn as nn
import math,time
from tianmoucv.reconstructor.integration import Reconstrutor_NN

def images_to_video(frame_list,name,size=(640,320),Flip=True):
    fps = 25        
    ftmax = max([np.max(ft) for ft in frame_list])
    ftmin = min([np.min(ft) for ft in frame_list])
    out = cv2.VideoWriter(name,0x7634706d , fps, size)
    for ft in frame_list:
        ft = (ft-ftmin)/(ftmax-ftmin)
        ft2 = (ft*255).astype(np.uint8)
        ft2 = ft2[:,::-1,:]
        out.write(ft2)
    out.release()

device = torch.device('cuda:2')
reconstructor = Reconstrutor_NN().to(device)
key_list = ["test_exam_fan_QRcode_3"]

for key in key_list:
    dataset = TianmoucDataReader(all_data,MAXLEN=500,ifSaveFileDict = False,matchkey=key,speedUpRate=1)
    dataLoader = torch.utils.data.DataLoader(dataset, batch_size=1,\
                                          num_workers=4, pin_memory=False, drop_last = False)
    img_list = []
    for index,sample in enumerate(dataLoader,0):
        if index<0:
            continue
        if index<= 5:
            F0 = sample['F0'][0,...]
            F1 = sample['F1'][0,...]
            tsdiff = sample['rawDiff'][0,...]/255.0
            #hdr = reconstructor.forward_single_t(F0,tsdiff,t) too slow
            
            reconstructed_b = reconstructor.forward_batch(F0.to(device),tsdiff.to(device)).float()

            for t in range(sample['rawDiff'].size(2)):
                reconstructed = reconstructed_b[t,...].cpu()
                reconstructed = reconstructed.permute(1,2,0).numpy()
                if t==12:
                    plt.figure(figsize=(12,4))
                    plt.subplot(1,3,1)  
                    plt.imshow(F1)
                    plt.subplot(1,3,2)  
                    plt.imshow(reconstructed)
                    plt.subplot(1,3,3)  
                    plt.imshow(F0)
                    plt.show()
                reconstructed[reconstructed>1]=1
                reconstructed[reconstructed<0]=0
                img_list.append(reconstructed)
        else:
            break
    images_to_video(img_list,'./Direct_RGB_'+key+'.mp4')

In [ ]:
!ls /home/lyh/anaconda3/envs/pytorch2/lib/python3.10/site-packages/tianmoucv/reconstructor/weight/